In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **PROJETO 3: COVID 19 MUNDIAL**

Este projeto analisa e implementa um algoritmo de Machine Learning para análise e previsão da evolução do Covid19 no período de 03/01/2020 a 12/08/2022.

Os dados foram retirados em 13/08/2022 da Organização Mundial da Saúde:

(https://covid19.who.int/table) 

## **ANÁLISE E EXPLORAÇÃO DOS DADOS**

In [ ]:
import numpy as np
import pandas as pd

path = '/content/drive/MyDrive/Udemy/Análise de Dados e Machine Learning/Seção 6/covid19_mundial.csv'
covid_mundial = pd.read_csv(path, sep=',')

In [ ]:
covid_mundial

In [ ]:
# Número de mortes acumuladas por país:

mortes_total = covid_mundial.groupby('Country').Cumulative_deaths.max()

# .groupby('Country') --> Agrupa por país
# .Cumulative_deaths.max()  --> Retorna o valor máximo das mortes acumuladas

In [ ]:
mortes_total.sort_values(ascending=False).head(20)

# ascending=False --> Ordem DECRESCENTE
# ascending=True --> Ordem CRESCENTE

In [ ]:
# Casos acumulados por país:

casos_total = covid_mundial.groupby('Country').Cumulative_cases.max()
casos_total.sort_values(ascending=False).head(20)

In [ ]:
covid_brasil = covid_mundial.loc[covid_mundial.Country == 'Brazil']

In [ ]:
covid_brasil

In [ ]:
covid_brasil.shape

In [ ]:
# Análise dos tipos de variáveis:

covid_brasil.dtypes

In [ ]:
# Análise dos valores missing:

covid_brasil.isnull().sum()

In [ ]:
covid_mundial.isnull().sum()

### **Análise de Valores Negativos**

In [ ]:
covid_mundial.loc[covid_mundial.New_cases < 0, :]

In [ ]:
# Contagem para cada variável:

covid_mundial.loc[covid_mundial.New_cases < 0, :].count()

In [ ]:
covid_mundial.loc[covid_mundial.New_deaths < 0, :].count()

In [ ]:
covid_brasil.loc[covid_brasil.New_cases < 0, :].count()

In [ ]:
covid_brasil.loc[covid_brasil.New_deaths < 0, :].count()

In [ ]:
# Criação de um DataFrame com a soma de todos os países por data:

covid_mundial_sum = covid_mundial.groupby('Date_reported').agg({'New_cases': 'sum', 
                                                                'Cumulative_cases': 'sum', 
                                                                'New_deaths': 'sum', 
                                                                'Cumulative_deaths': 'sum'})

# Soma todos os new_cases de cada país na data x e coloca na variável new_cases da nova tabela; o mesmo com as outras variáveis

In [ ]:
covid_mundial_sum

## **ANÁLISES ESTATÍSTICAS**

In [ ]:
covid_brasil.describe()

In [ ]:
covid_brasil.New_cases.mode()

In [ ]:
covid_brasil.New_deaths.mode()

### **Análise dos Outliers**

In [ ]:
import plotly.express as px

In [ ]:
px.box(covid_brasil, y='Cumulative_cases')

In [ ]:
px.box(covid_brasil, y='Cumulative_deaths')

### **Análise de Normalidade**

In [ ]:
import seaborn as sns

In [ ]:
sns.histplot(covid_brasil, x='Cumulative_cases', bins=20, color='brown', kde=True, stat='count',);

In [ ]:
sns.histplot(covid_brasil, x='Cumulative_deaths', bins=20, color='brown', kde=True, stat='count',);

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt

In [ ]:
stats.probplot(covid_brasil['Cumulative_cases'], dist='norm', plot=plt)
plt.title('Análise Normalidade')
plt.show()

In [ ]:
stats.probplot(covid_brasil['Cumulative_deaths'], dist='norm', plot=plt)
plt.title('Análise Normalidade')
plt.show()

In [ ]:
import statsmodels
from statsmodels.stats.diagnostic import lilliefors

In [ ]:
estatistica, p = statsmodels.stats.diagnostic.lilliefors(covid_brasil.Cumulative_cases, dist='norm',)
print(f'Estatística do teste (D) = {estatistica}')
print(f'p_valor = {p}')

In [ ]:
estatistica, p = statsmodels.stats.diagnostic.lilliefors(covid_brasil.Cumulative_deaths, dist='norm',)
print(f'Estatística do teste (D) = {estatistica}')
print(f'p_valor = {p}')

### **Gráfico de dispersão em função das datas**

In [ ]:
import plotly.express as px

In [ ]:
disp = px.scatter(x=covid_brasil.Date_reported, y=covid_brasil.Cumulative_cases)
disp.update_layout(width=900, height=400, title_text='Número de Casos Acumulados no Brasil')
disp.update_xaxes(title='Datas')
disp.update_yaxes(title='Casos')
disp.show()

In [ ]:
disp = px.scatter(x=covid_brasil.Date_reported, y=covid_brasil.Cumulative_deaths)
disp.update_layout(width=900, height=400, title_text='Número de Mortes Acumuladas no Brasil')
disp.update_xaxes(title='Datas')
disp.update_yaxes(title='Mortes')
disp.show()

In [ ]:
plt.subplots(figsize=(10,5))
plt.stackplot(covid_brasil['Date_reported'], covid_brasil['Cumulative_cases'], covid_brasil['Cumulative_deaths'],
              labels=['Cumulative_cases', 'Cumulatives_deaths'])
plt.legend(loc = 'upper left')  # Adiciona legenda no gráfico na posição especificada
plt.title('Comparação da evolução dos casos e das mortes no Brasil');

### **Correlação Linear**

In [ ]:
correlacoes = covid_brasil.corr(method='spearman')
correlacoes

In [ ]:
plt.figure()
sns.heatmap(correlacoes, annot=True);

## **MACHINE LEARNING**

### **Regressão Linear Simples**

In [ ]:
covid_brasil

In [ ]:
covid_brasil.shape

In [ ]:
import plotly.express as px

disp = px.scatter(x=covid_brasil.New_cases, y=covid_brasil.New_deaths)
disp.update_layout(width=900, height=400, title_text='Número de mortes em função do número de casos')
disp.update_xaxes(title='Novos Casos')
disp.update_yaxes(title='Novas Mortes')
disp.show()

In [ ]:
# variavel_x_y = nome_tabela.iloc[:, indice_coluna].values

x = covid_brasil.iloc[:, 4].values  # Novos casos  (índice 4)
y = covid_brasil.iloc[:, 6].values  # Novas mortes (índice 6)

In [ ]:
x

In [ ]:
y

In [ ]:
# Transoformar em matriz:
x = x.reshape(-1, 1)

In [ ]:
x

**Separando base de dados em teste e treinamento**

In [ ]:
from sklearn.model_selection import train_test_split

# x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(variavel_x, variavel_y, 
#                                                                  test_size = tamanho_teste,
#                                                                  random_state = semente_aleatória)

# Sem o random_state, o resultado não se repetirá

x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, 
                                                                  test_size = 0.25,  # 25% para teste e 75% para treinamento
                                                                  random_state = 2)

In [ ]:
x_treinamento

In [ ]:
x_treinamento.size

In [ ]:
x_teste

In [ ]:
x_teste.size

In [ ]:
y_treinamento

In [ ]:
y_treinamento.size

In [ ]:
y_teste

In [ ]:
y_teste.size

**Criação do modelo de regressão linear**

In [ ]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(x_treinamento, y_treinamento)
score = regressor.score(x_treinamento, y_treinamento)

# Score --> Coeficiente de determinação

In [ ]:
# Gráfico com dados de treinamento:

import matplotlib.pyplot as plt

plt.scatter(x_treinamento, y_treinamento)
plt.plot(x_treinamento, regressor.predict(x_treinamento), color='red');

In [ ]:
score

In [ ]:
previsoes = regressor.predict(x_teste)

In [ ]:
# Gráfico com os dados de teste:

plt.scatter(x_teste, y_teste)
plt.plot(x_teste, regressor.predict(x_teste), color='red');

In [ ]:
previsao = regressor.predict(np.array(80000).reshape(-1,1))
previsao

# Tendo 80000 casos, a previsão é ter 1218 mortes.

In [ ]:
# Coeficiente linear:

regressor.intercept_

In [ ]:
# Coeficiente angular:

regressor.coef_

Equação: mortes = 313.06751 + 0.0113*casos

**Métricas de Desempenho**

In [ ]:
# Coeficiente de Determinação (R^2):

regressor.score(x_teste, y_teste)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

print(f'Erro Médio Absoluto (MAE): {mean_absolute_error(y_teste, previsoes)}')
print(f'Erro Quadrático Médio (MSE): {mean_squared_error(y_teste, previsoes)}')
print(f'Raiz do Erro Quadrático Médio (RMSE): {np.sqrt(mean_squared_error(y_teste, previsoes))}')

# Quanto menores os resultados, melhor o algoritmo

### **Regressão Polinomial**

In [ ]:
covid_brasil

In [ ]:
import plotly.express as px

disp = px.scatter(x=covid_brasil.Date_reported, y=covid_brasil.Cumulative_cases)
disp.update_layout(width=900, height=400, title_text='Número de Casos Acumulados no Brasil')
disp.update_xaxes(title='Datas')
disp.update_yaxes(title='Casos')
disp.show()

In [ ]:
x = covid_brasil.iloc[:, 0].values  #     Datas        (índice 0)
y = covid_brasil.iloc[:, 5].values  # Casos Acumulados (índice 5)

In [ ]:
x

In [ ]:
# Transformar as datas em sequência numérica (1,2,3,4,...) e em matriz:

x = np.arange(1, len(x) + 1).reshape(-1,1)
x

In [ ]:
y

**Separando base de dados em teste e treinamento**

In [ ]:
from sklearn.model_selection import train_test_split

x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, 
                                                                  test_size = 0.25,
                                                                  shuffle = True,
                                                                  random_state = 2,)

In [ ]:
x_treinamento.size

In [ ]:
x_teste.size

In [ ]:
y_treinamento.size

In [ ]:
y_teste.size

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)  # De grau 2
x_treinamento_poly = poly.fit_transform(x_treinamento)
x_teste_poly = poly.transform(x_teste)

In [ ]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(x_treinamento_poly, y_treinamento)
score = regressor.score(x_treinamento_poly, y_treinamento)

In [ ]:
y_treinamento.size

In [ ]:
previsoes = regressor.predict(x_teste_poly)

In [ ]:
previsoes.size

In [ ]:
# Criação de uma sequência para previsão:

forecast = np.arange(len(x) + 20).reshape(-1,1)  # + 20 é uma previsão de 20 dias a mais

In [ ]:
forecast.shape

In [ ]:
x_train_total = poly.transform(forecast)
x_train_total.shape

In [ ]:
x_train_total

In [ ]:
previsao_total = regressor.predict(x_train_total)
len(previsao_total)

In [ ]:
previsao_total

In [ ]:
plt.subplots(figsize=(10,5))
plt.plot(forecast[:-20], y, color='red')
plt.plot(forecast, previsao_total, linestyle='dashed')
plt.title('Casos de COVID-19 no Brasil')
plt.xlabel('A partir de 03/01/2020')
plt.ylabel('Número de Casos')
plt.legend(['Casos Acumulados', 'Previsão']);

In [ ]:
previsao_total[950]

**Métricas de Desempenho**

In [ ]:
# Coeficiente de Determinação (R^2)

score

In [ ]:
poly_teste_pred = regressor.predict(x_teste_poly)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

print(f'Erro Médio Absoluto (MAE): {mean_absolute_error(poly_teste_pred, y_teste)}')
print(f'Erro Quadrático Médio (MSE): {mean_squared_error(poly_teste_pred, y_teste)}')
print(f'Raiz do Erro Quadrático Médio (RMSE): {np.sqrt(mean_squared_error(poly_teste_pred, y_teste))}')